# Conversational agent

- [1. Set OpenAI API Key](#1. Set OpenAI-API-Key)
- [2. Build Agent step by step](#2. Build Agent step by step)
- [2.1 Define Tools and Function](##2.1-Define Tools and Function)
- [2.2 Add intermediate step results](##2.2-Add intermediate step results)
- [2.3 Add conversation history](##2.3-Add conversation history)
- [3. Create a conversation robot](#3. Create a conversation robot)
- [4. English version prompts](#4. English version prompts)

# 1. Set OpenAI-API-Key

For details, see `Set OpenAI_API_KEY.ipynb` file

# 2. Build Agent step by step

## 2.1 Define Tools and Function

In [30]:
# Import the tool package
from langchain.tools import tool

In [31]:
# Import required libraries
import requests
from pydantic import BaseModel, Field
import datetime

# Define input format
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="要获取天气数据的位置的纬度") 
    longitude: float = Field(..., description="要获取天气数据的位置的经度") 

# Use the @tool decorator and specify the input format
@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
""""Get the temperature at the given coordinate position"""
    
# URL of the Open Meteo API
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
# Request Parameters
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

# Sending API Requests
    response = requests.get(BASE_URL, params=params)
    
# Check the response status code
    if response.status_code == 200:
# Parsing the JSON response
        results = response.json()
    else:
# Handling request failures
        raise Exception(f"API Request failed with status code: {response.status_code}")

# Get the current UTC time
    current_utc_time = datetime.datetime.utcnow()
    
# Convert a time string to a datetime object
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    
# Get the temperature list
    temperature_list = results['hourly']['temperature_2m']
    
# Find the index closest to the current time
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    
# Get the current temperature
    current_temperature = temperature_list[closest_time_index]
    
# Returns the current temperature as a string
    return f'现在温度是 {current_temperature}°C'


In [32]:
import wikipedia

# Define the tool for Wikipedia search
@tool
def search_wikipedia(query: str) -> str:
"""Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]: #取前三个页面标题
        try:
#Use the page function of the wikipedia module to get the Wikipedia page object for the specified title.
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False) 
# Get page summary
            summaries.append(f"页面: {page_title}\n摘要: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "维基百科没有搜索到合适的结果"
    return "\n\n".join(summaries)

In [33]:
# Add to tools list
tools = [get_current_temperature, search_wikipedia]

In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [35]:
# Format tools as OpenAI functions
functions = [format_tool_to_openai_function(f) for f in tools]

# Create a ChatOpenAI model, set the temperature to 0, and bind the generated OpenAI function list
model = ChatOpenAI(temperature=0).bind(functions=functions)

# Create ChatPromptTemplate to get user input from the message template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])

# Create a processing chain to connect prompt, model and OpenAIFunctionsAgentOutputParser
chain = prompt | model | OpenAIFunctionsAgentOutputParser()


In [36]:
# transfer
result = chain.invoke({"input": "现在圣佛朗西斯科的温度是多少?"})

In [37]:
# View the called tools
result.tool

'get_current_temperature'

In [38]:
# View tool input
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [39]:
result.tool

'get_current_temperature'

In [40]:
result.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

## 2.2 Add intermediate step results

In [41]:
# Create ChatPromptTemplate to get user input from the message template
from langchain.prompts import MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个安全且乐于助人的助手"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad") # 解析和验证关键字参数中的输入数据
])

In [42]:
# Create a processing chain to connect prompt, model and OpenAIFunctionsAgentOutputParser
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [43]:
# transfer
result1 = chain.invoke({
    "input": "现在圣佛朗西斯科的温度是多少?",
    "agent_scratchpad": []
})

In [44]:
# View the called tools
result1.tool

'get_current_temperature'

In [45]:
# Get the result of tool call
observation = get_current_temperature(result1.tool_input)

In [46]:
# View Results
observation

'现在温度是 9.2°C'

In [47]:
# Check the output type
type(result1)

langchain_core.agents.AgentActionMessageLog

In [48]:
from langchain.agents.format_scratchpad import format_to_openai_functions

In [49]:
# View call log
result1.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}})]

In [50]:
# Convert the results of Function and Tool to openai function format
format_to_openai_functions([(result1, observation), ])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}}),
 FunctionMessage(content='现在温度是 9.2°C', name='get_current_temperature')]

In [51]:
# Construct the input format of the Agent, agent_scratchpad is the result of the intermediate step
result2 = chain.invoke({
    "input": "现在圣佛朗西斯科的温度是多少?", 
    "agent_scratchpad": format_to_openai_functions([(result1, observation)])
})

In [52]:
# View the output of the Agent
result2

AgentFinish(return_values={'output': '圣佛朗西斯科现在的温度是9.2°C。'}, log='圣佛朗西斯科现在的温度是9.2°C。')

In [53]:
from langchain.schema.agent import AgentFinish

def run_agent(user_input):
# Store the list of intermediate steps
    intermediate_steps = []

# Loop through agent operations
    while True:
# Call the invoke method of the processing chain, passing the user input and the OpenAI function form of the intermediate step
        result = chain.invoke({
            "input": user_input,
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })

# If the result is of type AgentFinish, the run ends and the result is returned
        if isinstance(result, AgentFinish):
            return result

# Select the corresponding tool function according to the tool name in the result
        tool = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }[result.tool]

# Run the selected tool function and obtain the observation results
        observation = tool.run(result.tool_input)

# Add results and observations to the list of intermediate steps
        intermediate_steps.append((result, observation))


In [54]:
from langchain.schema.runnable import RunnablePassthrough

# Create a RunnablePassthrough to convert the intermediate step results stored in agent_scratchpad into OpenAI function form
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain


In [55]:
# Define a function to run the agent
def run_agent(user_input):
# Store the list of intermediate steps
    intermediate_steps = []

# Loop through agent operations
    while True:
# Call the invoke method through agent_chain, passing the user input and the list of intermediate steps
        result = agent_chain.invoke({
            "input": user_input,
            "intermediate_steps": intermediate_steps
        })

# If the result is of type AgentFinish, end the loop and return the result
        if isinstance(result, AgentFinish):
            return result

# Select the corresponding tool function according to the tool name in the result
        tool = {
            "search_wikipedia": search_wikipedia,
            "get_current_temperature": get_current_temperature,
        }[result.tool]

# Run the selected tool function and obtain the observation results
        observation = tool.run(result.tool_input)

# Add results and observations to the list of intermediate steps
        intermediate_steps.append((result, observation))


In [56]:
run_agent("圣弗朗西斯科的温度是多少？")

AgentFinish(return_values={'output': '圣弗朗西斯科现在的温度是9.2°C。'}, log='圣弗朗西斯科现在的温度是9.2°C。')

In [33]:
run_agent("什么是langchain?")

AgentFinish(return_values={'output': 'LangChain是一个旨在简化使用大型语言模型（LLMs）创建应用程序的框架。作为一个语言模型集成框架，LangChain的用例主要涵盖了文档分析和摘要、聊天机器人以及代码分析等领域。'}, log='LangChain是一个旨在简化使用大型语言模型（LLMs）创建应用程序的框架。作为一个语言模型集成框架，LangChain的用例主要涵盖了文档分析和摘要、聊天机器人以及代码分析等领域。')

In [35]:
run_agent("你好！")

AgentFinish(return_values={'output': '你好！有什么可以帮助你的吗？'}, log='你好！有什么可以帮助你的吗？')

In [57]:
# Use AgentExecutor to encapsulate the agent
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [38]:
agent_executor.invoke({"input": "什么是langchain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'Langchain'}`


Page: LangChain
Summary: LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: OpenAI
Summary: OpenAI is a U.S. based artificial intelligence (AI) research organization founded in December 2015, researching artificial intelligence with the goal of developing "safe and beneficial" artificial general intelligence, which it defines as "highly autonomous systems that outperform humans at most economically valuable work".
As one of the leading organizations of the AI spring, it has developed several large language models, advanced image generation models, and previously, released open-source models. Its release of ChatGPT has been c

{'input': '什么是langchain?',
 'output': 'LangChain 是一个旨在简化使用大型语言模型（LLMs）创建应用程序的框架。作为一个语言模型集成框架，LangChain 的用例主要涵盖了文档分析和摘要、聊天机器人以及代码分析等领域。'}

In [40]:
agent_executor.invoke({"input": "我的名字是bob"})



> Entering new AgentExecutor chain...
你好，Bob！有什么可以帮助你的吗？

> Finished chain.


{'input': '我的名字是bob', 'output': '你好，Bob！有什么可以帮助你的吗？'}

In [42]:
agent_executor.invoke({"input": "我的名字是什么？"})



> Entering new AgentExecutor chain...
你的名字是用户。

> Finished chain.


{'input': '我的名字是什么？', 'output': '你的名字是用户。'}

## 3.3 Adding conversation history

In [62]:
# Add the user's conversation history
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [63]:
# Construct the agent's processing chain
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [64]:
# Add dialogue memory module
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [65]:
# Encapsulate AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [48]:
agent_executor.invoke({"input": "我的名字是bob"})



> Entering new AgentExecutor chain...
Nice to meet you, Bob! How can I assist you today?

> Finished chain.


{'input': '我的名字是bob',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='我的名字是bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?')],
 'output': 'Nice to meet you, Bob! How can I assist you today?'}

In [50]:
agent_executor.invoke({"input": "我的名字是什么？"})



> Entering new AgentExecutor chain...
你的名字是 Bob。有什么我可以帮助你的吗？

> Finished chain.


{'input': '我的名字是什么？',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='我的名字是bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob. How can I assist you today, Bob?'),
  HumanMessage(content='我的名字是什么？'),
  AIMessage(content='你的名字是 Bob。有什么我可以帮助你的吗？')],
 'output': '你的名字是 Bob。有什么我可以帮助你的吗？'}

In [52]:
agent_executor.invoke({"input": "圣弗朗西斯科现在的温度是多少？"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`


The current temperature is 9.1°CThe current temperature in San Francisco is 9.1°C. Is there anything else you would like to know?

> Finished chain.


{'input': '圣弗朗西斯科现在的温度是多少？',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='我的名字是bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='whats my name'),
  AIMessage(content='Your name is Bob. How can I assist you today, Bob?'),
  HumanMessage(content='我的名字是什么？'),
  AIMessage(content='你的名字是 Bob。有什么我可以帮助你的吗？'),
  HumanMessage(content='whats the weather in sf?'),
  AIMessage(content='The current temperature in San Francisco is 9.1°C. Is there anything else you would like to know?'),
  HumanMessage(content='圣弗朗西斯科现在的温度是多少？'),
  AIMessage(content='The current temperature in San Francisco is 9.1°C. Is there anything else you would like to know?')],
 'output': 'The current temperature in San Francisco is 9.1°C. Is there anything else you would like to know?'}

# 3. Create a conversational robot

In [58]:
# Custom tools (free play)
@tool
def create_your_own(query: str) -> str:
"""Customizable functions """
    print(type(query))
    return query[::-1]

In [59]:
# Add the previously defined tools to the tool list
tools = [get_current_temperature, search_wikipedia, create_your_own]

In [60]:
import panel as pn  # GUI
pn.extension()
import panel as pn
import param

# Define the cbfs class
class cbfs(param.Parameterized):
    
# Initialization function
    def __init__(self, tools, **params):
        super(cbfs, self).__init__(**params)
        self.panels = []  # 存储 GUI 面板
        self.functions = [format_tool_to_openai_function(f) for f in tools]  # 将tools格式化为 OpenAI 函数
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)  # 创建 ChatOpenAI 模型
        self.memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")  # 创建 ConversationBufferMemory
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])  # 创建 ChatPromptTemplate
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()  # 创建处理链
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)  # 创建 AgentExecutor
    
# Dialogue chain function
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)

# Clear history function
    def clr_history(self, count=0):
        self.chat_history = []
        return


In [67]:
# Create a cbfs object, passing a list of tools
cb = cbfs(tools)

# Create a text input box component
inp = pn.widgets.TextInput(placeholder='请输入文本...')

# Bind the convchain method to the input box to form a dialogue chain
conversation = pn.bind(cb.convchain, inp) 

# Create the first tab component
tab1 = pn.Column(
    pn.Row(inp),  # 显示文本输入框
    pn.layout.Divider(),
    pn.panel(conversation, loading_indicator=True, height=400),  # 显示对话链面板
    pn.layout.Divider(),
)

# Create a dashboard, including a title and tabs
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# 对话机器人')),  # 显示标题
    pn.Tabs(('发送', tab1))  # 创建选项卡
)

# Display the dashboard
dashboard


Column
    [0] Row
        [0] Markdown(str)
    [1] Tabs
        [0] Column
            [0] Row
                [0] TextInput(placeholder='请输入文本...')
            [1] Divider()
            [2] ParamFunction(function, _pane=Str, defer_load=False, height=400, loading_indicator=True)
            [3] Divider()

The chatbot effect is shown below:

![](../../figures/chat_bot.png)

# 4. English version tips

To summarize the process of building a conversational robot:

Define tools and functions --> Build a model --> Add intermediate step results --> Add a memory mechanism --> Interactive interface

In [47]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [48]:
# Construct the agent's processing chain
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [49]:
# Add dialogue memory module
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [50]:
# Encapsulate AgentExecutor
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [51]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Nice to meet you, Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?')],
 'output': 'Nice to meet you, Bob! How can I assist you today?'}

In [52]:
agent_executor.invoke({"input": "what is my name"})



> Entering new AgentExecutor chain...
Your name is Bob. How can I assist you today, Bob?

> Finished chain.


{'input': 'what is my name',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='what is my name'),
  AIMessage(content='Your name is Bob. How can I assist you today, Bob?')],
 'output': 'Your name is Bob. How can I assist you today, Bob?'}

In [53]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`


现在温度是 9.5°CThe current temperature in San Francisco is 9.5°C.

> Finished chain.


{'input': 'whats the weather in sf?',
 'chat_history': [HumanMessage(content='my name is bob'),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?'),
  HumanMessage(content='what is my name'),
  AIMessage(content='Your name is Bob. How can I assist you today, Bob?'),
  HumanMessage(content='whats the weather in sf?'),
  AIMessage(content='The current temperature in San Francisco is 9.5°C.')],
 'output': 'The current temperature in San Francisco is 9.5°C.'}